## Description: prepare the notebook for manual annotation

In this notebook, three elements are added to the previously generated dataset:
- check that the chapter and verse_num columns are integers
- sign_info column, based on a recent version of the function
- reconstructed_verse column which is a gcons version of the verse with brackets indicating the reconstructed sections
- the spatial_arg_type column for annotating goal, source, location, etc. 
- remove the rows which have "MN" in preposition_1 and no preposition_2

In [25]:
import pandas as pd

from itertools import chain

from tfob import TFOb,  get_dss

DSS = get_dss()

**Locating corpus resources ...**

Name,# of nodes,# slots / node,% coverage
scroll,1001,1428.81,100
lex,10450,129.14,94
fragment,11182,127.91,100
line,52895,27.04,100
clause,125,12.85,0
cluster,101099,6.68,47
phrase,315,5.10,0
word,500995,2.81,99
sign,1430241,1.00,100


In [27]:
PATH = 'data/biblical_datasets/proverbs/annotation_df_history/proverbs_dataset.csv' # Modify before executing the notebook

In [28]:
#df_1 = pd.read_csv('../2_datasets/generate_datasets/data/biblical_datasets/Jeremiah_dataset.csv').fillna("")
df = pd.read_csv(PATH).fillna("")

In [29]:
# Make the values in "chapter" and "verse_num" integers
df['chapter'] = df['chapter'].astype(int)
df['verse_num'] = df['verse_num'].astype(int)

In [30]:
# Reorganise the dataset by chapter, verse_num and scroll

# Custom sorting logic for "scroll" column
df['scroll_order'] = df['scroll'].apply(lambda x: 0 if x == "MT" else 1)

# Sort by "scroll_order", "chapter", and "verse_num"
df = df.sort_values(by=['chapter', 'verse_num', 'scroll_order'])

# Drop the temporary 'scroll_order' column
df = df.drop(columns='scroll_order')

df = df.reset_index(drop=True)

In [31]:
pd.set_option("display.max_columns", None)

In [32]:
df.head(20)

,verb_id,lex,scroll,book,chapter,verse_num,gcons_verb,gcons_verse,gcons_clause,subject,complement,cmpl_lex,cmpl_translation,dir_he,cmpl_constr,cmpl_nt,cmpl_anim,cmpl_det,cmpl_indiv,cmpl_complex,dir_he_dss,sign_info,stem,tense,motion_type,comments,preposition_1,preposition_2,preposition_3,preposition_4,preposition_5,preposition_6,preposition_7
0,347025,HLK[,MT,Proverbs,1,11,LKH,>M J>MRW LKH >TNW N>RBH L DM NYPNH L NQJ XNM,LKH >TNW,,>TNW,>T==,,0.0,prep,prsf,,det,prsf,simple,,,qal,impv,,,>T==,,,,,,
1,347051,NPL[,MT,Proverbs,1,14,TPJL,GWRLK TPJL B TWKNW KJS >XD JHJH L KLNW,GWRLK TPJL B TWKNW,,B TWKNW,B TWK/,,0.0,prep,,,det,subs,complex,,,hif,impf,,,B,,,,,,
2,347061,HLK[,MT,Proverbs,1,15,TLK,BNJ >L TLK B DRK >TM MN< RGLK M NTJBTM,>L TLK B DRK >TM,,B DRK,B DRK/,,0.0,prep,,,und,subs,simple,,,qal,impf,,,B,,,,,,
3,347074,RWY[,MT,Proverbs,1,16,JRWYW,KJ RGLJHM L R< JRWYW W JMHRW L CPK DM,KJ RGLJHM L R< JRWYW,RGLJHM,L R<,L H R</,,0.0,prep,,,det,,simple,,,qal,impf,,,L,,,,,,
4,347076,MHR[,MT,Proverbs,1,16,JMHRW,KJ RGLJHM L R< JRWYW W JMHRW L CPK DM,W JMHRW,,no complement,no complement,,,,,,,,,,,piel,impf,,,,,,,,,
5,347143,CWB[,MT,Proverbs,1,23,TCWBW,TCWBW L TWKXTJ HNH >BJ<H LKM RWXJ >WDJ<H DBRJ ...,TCWBW L TWKXTJ,,L TWKXTJ,L TWKXT/,,0.0,prep,,,det,subs,complex,,,qal,impf,,,L,,,,,,
6,347177,BW>[,MT,Proverbs,1,26,B>,GM >NJ B >JDKM >FXQ >L<G B B> PXDKM,B B> PXDKM,PXDKM,no complement,no complement,,,,,,,,,,,qal,infc,,,,,,,,,
7,347180,BW>[,MT,Proverbs,1,27,B>,B B> K C>WH PXDKM W >JDKM K SWPH J>TH B B> <LJ...,B B> K C>WH PXDKM,PXDKM,no complement,no complement,,,,,,,,,,,qal,infc,,,,,,,,,
8,347188,>TH[,MT,Proverbs,1,27,J>TH,B B> K C>WH PXDKM W >JDKM K SWPH J>TH B B> <LJ...,W >JDKM K SWPH J>TH,>JDKM,no complement,no complement,,,,,,,,,,,qal,impf,,,,,,,,,
9,347190,BW>[,MT,Proverbs,1,27,B>,B B> K C>WH PXDKM W >JDKM K SWPH J>TH B B> <LJ...,B B> <LJKM YRH W YWQH,YRH W YWQH,<LJKM,<L,,0.0,prep,prsf,,det,prsf,simple,,,qal,infc,,,<L,,,,,,


In [33]:
df[(df.scroll != "MT") & (df.gcons_clause != "None")]

,verb_id,lex,scroll,book,chapter,verse_num,gcons_verb,gcons_verse,gcons_clause,subject,complement,cmpl_lex,cmpl_translation,dir_he,cmpl_constr,cmpl_nt,cmpl_anim,cmpl_det,cmpl_indiv,cmpl_complex,dir_he_dss,sign_info,stem,tense,motion_type,comments,preposition_1,preposition_2,preposition_3,preposition_4,preposition_5,preposition_6,preposition_7
65,2055315,HLK[,4Q103,Proverbs,14,7,LK,LK M NGD L >JC KSJL W BL JD<T FPTJ D<T,LK M NGD L >JC KSJL,,M NGD L >JC KSJL,,,0.0,,,,,,,0.0,,qal,impv,,,MN,L,,,,,
68,2055406,NWX[,4Q103,Proverbs,14,33,TNWX,B LB NBWN TNWX XKMH W B QRB KSJLJM TWD<,B LB NBWN TNWX XKMH,XKMH,B LB NBWN,,,0.0,,,,,,,0.0,,qal,impf,,,B,,,,,,


In [34]:
df.columns

Index(['verb_id', 'lex', 'scroll', 'book', 'chapter', 'verse_num',
       'gcons_verb', 'gcons_verse', 'gcons_clause', 'subject', 'complement',
       'cmpl_lex', 'cmpl_translation', 'dir_he', 'cmpl_constr', 'cmpl_nt',
       'cmpl_anim', 'cmpl_det', 'cmpl_indiv', 'cmpl_complex', 'dir_he_dss',
       'sign_info', 'stem', 'tense', 'motion_type', 'comments',
       'preposition_1', 'preposition_2', 'preposition_3', 'preposition_4',
       'preposition_5', 'preposition_6', 'preposition_7'],
      dtype='object')

In [35]:
# This is the most recent sign_info function which uses the .rec (1 October 2024)

def sign_info(verb):
    sign_rec_words = []
    
    scroll = verb.to_scrolls.scroll[0]
    verse = TFOb.section([verb.book[0], verb.chapter[0], verb.verse[0]], DSS, scroll)
    
    for word in verse.to_words:
        signs = word.to_signs
        sign_rec = []
        for sign in signs:
            if sign.type[0] == "cons": # only keeps consonants
                if sign.rec[0]:
                    sign_rec.append("1")
                else:
                    sign_rec.append("0")
        if sign_rec:
            sign_rec_words.append("".join(sign_rec))
        
    return " ".join(sign_rec_words).strip()

In [36]:
# Updates the dataframe (change the old sign_info column to one containing the return of the new function)

def update_sign_info(row):
    if row['scroll'] != 'MT':
        return sign_info(TFOb(int(row['verb_id']), DSS))
    return row['sign_info']

# Apply the function to the dataframe
df['sign_info'] = df.apply(update_sign_info, axis=1)

In [37]:
# Initialize an empty list to collect mismatch information
mismatches = []

# Function to check the lengths of sign_info and gcons_verse
def check_length(row):
    if row['scroll'] != 'MT':
        if len(row['sign_info']) != len(row['gcons_verse']):
            mismatches.append(f"Mismatch found in row {row.name}: sign_info='{row['sign_info']}' (len={len(row['sign_info'])}), gcons_verse='{row['gcons_verse']}' (len={len(row['gcons_verse'])})")

# Apply the function to the dataframe
df.apply(check_length, axis=1)

# Check if there are any mismatches and print them
if mismatches:
    print("Mismatches found:")
    for mismatch in mismatches:
        print(mismatch)
else:
    print("No mismatches found.")

No mismatches found.


In [38]:
# The reconstruct_verse function adds a gcons version of the verse with brackets indicating the reconstructed sections

# Add a column with the verse plus brackets indicating reconstructed letters

def reconstruct_verse(sign_info, verse):
    rec_verse = []
    is_reconstructed = False

    for c_verse, c_sign in zip(verse, sign_info):
        if c_sign == "1" and not is_reconstructed:
            is_reconstructed = True
            rec_verse.append("[")
        if c_sign == "0" and is_reconstructed:
            is_reconstructed = False
            if rec_verse[-1] == " ":
                rec_verse.pop()
                rec_verse.extend(["]", " "])
            else:
                rec_verse.append("]")
        rec_verse.append(c_verse)

    if is_reconstructed:
        rec_verse.append("]")
    return "".join(rec_verse)


df["reconstructed_verse"] = ""

df["reconstructed_verse"] = df.apply(lambda row: reconstruct_verse(row.sign_info, row.gcons_verse) if row.book != "MT" else "", axis=1)

In [39]:
# Delete the DSS duplicated

verb_ids = df.verb_id.to_list()

duplicates = set([verb_id for verb_id in verb_ids if verb_ids.count(verb_id) > 1])


df = df[~((df.gcons_clause == "None") & (df.subject == "None") & (df.verb_id.isin(duplicates)))] # delete duplicates
df.reset_index(drop=True, inplace=True) # reset the index for AnnotEasy

In [40]:
df.columns

Index(['verb_id', 'lex', 'scroll', 'book', 'chapter', 'verse_num',
       'gcons_verb', 'gcons_verse', 'gcons_clause', 'subject', 'complement',
       'cmpl_lex', 'cmpl_translation', 'dir_he', 'cmpl_constr', 'cmpl_nt',
       'cmpl_anim', 'cmpl_det', 'cmpl_indiv', 'cmpl_complex', 'dir_he_dss',
       'sign_info', 'stem', 'tense', 'motion_type', 'comments',
       'preposition_1', 'preposition_2', 'preposition_3', 'preposition_4',
       'preposition_5', 'preposition_6', 'preposition_7',
       'reconstructed_verse'],
      dtype='object')

In [41]:
# Add the spatial_arg_type column

df["spatial_arg_type"] = ""

In [42]:
df.columns

Index(['verb_id', 'lex', 'scroll', 'book', 'chapter', 'verse_num',
       'gcons_verb', 'gcons_verse', 'gcons_clause', 'subject', 'complement',
       'cmpl_lex', 'cmpl_translation', 'dir_he', 'cmpl_constr', 'cmpl_nt',
       'cmpl_anim', 'cmpl_det', 'cmpl_indiv', 'cmpl_complex', 'dir_he_dss',
       'sign_info', 'stem', 'tense', 'motion_type', 'comments',
       'preposition_1', 'preposition_2', 'preposition_3', 'preposition_4',
       'preposition_5', 'preposition_6', 'preposition_7',
       'reconstructed_verse', 'spatial_arg_type'],
      dtype='object')

In [43]:
# Remove the columns that have MN as preposition 1 and not other prepositions

In [44]:
# have a look at the data
df_test = df[(df.preposition_1 == "MN") & (df.preposition_2 == "MN")]
df_test.head(5)
#len(df_test)

,verb_id,lex,scroll,book,chapter,verse_num,gcons_verb,gcons_verse,gcons_clause,subject,complement,cmpl_lex,cmpl_translation,dir_he,cmpl_constr,cmpl_nt,cmpl_anim,cmpl_det,cmpl_indiv,cmpl_complex,dir_he_dss,sign_info,stem,tense,motion_type,comments,preposition_1,preposition_2,preposition_3,preposition_4,preposition_5,preposition_6,preposition_7,reconstructed_verse,spatial_arg_type


In [45]:
len(df_test)

0

In [46]:
# Identify the rows to drop based on the condition
index_to_drop = df[(df.preposition_1 == "MN") & (df.preposition_2 == "")].index
index_to_drop_2 = df[(df.preposition_1 == "MN") & (df.preposition_2 == "MN") & (df.preposition_3 == "")].index
index_to_drop_3 = df[(df.preposition_1 == "MN") & (df.preposition_2 == "MN") & (df.preposition_3 == "MN") & (df.preposition_4 == "")].index

# Drop the rows by index
df = df.drop(index_to_drop)
df = df.drop(index_to_drop_2)
df = df.drop(index_to_drop_3)

# Reset the index
df = df.reset_index(drop=True)

In [47]:
# Verification
df[(df.preposition_1 == "MN") & (df.preposition_2 == "")]

,verb_id,lex,scroll,book,chapter,verse_num,gcons_verb,gcons_verse,gcons_clause,subject,complement,cmpl_lex,cmpl_translation,dir_he,cmpl_constr,cmpl_nt,cmpl_anim,cmpl_det,cmpl_indiv,cmpl_complex,dir_he_dss,sign_info,stem,tense,motion_type,comments,preposition_1,preposition_2,preposition_3,preposition_4,preposition_5,preposition_6,preposition_7,reconstructed_verse,spatial_arg_type


In [48]:
df.head(20)

,verb_id,lex,scroll,book,chapter,verse_num,gcons_verb,gcons_verse,gcons_clause,subject,complement,cmpl_lex,cmpl_translation,dir_he,cmpl_constr,cmpl_nt,cmpl_anim,cmpl_det,cmpl_indiv,cmpl_complex,dir_he_dss,sign_info,stem,tense,motion_type,comments,preposition_1,preposition_2,preposition_3,preposition_4,preposition_5,preposition_6,preposition_7,reconstructed_verse,spatial_arg_type
0,347025,HLK[,MT,Proverbs,1,11,LKH,>M J>MRW LKH >TNW N>RBH L DM NYPNH L NQJ XNM,LKH >TNW,,>TNW,>T==,,0.0,prep,prsf,,det,prsf,simple,,,qal,impv,,,>T==,,,,,,,,
1,347051,NPL[,MT,Proverbs,1,14,TPJL,GWRLK TPJL B TWKNW KJS >XD JHJH L KLNW,GWRLK TPJL B TWKNW,,B TWKNW,B TWK/,,0.0,prep,,,det,subs,complex,,,hif,impf,,,B,,,,,,,,
2,347061,HLK[,MT,Proverbs,1,15,TLK,BNJ >L TLK B DRK >TM MN< RGLK M NTJBTM,>L TLK B DRK >TM,,B DRK,B DRK/,,0.0,prep,,,und,subs,simple,,,qal,impf,,,B,,,,,,,,
3,347074,RWY[,MT,Proverbs,1,16,JRWYW,KJ RGLJHM L R< JRWYW W JMHRW L CPK DM,KJ RGLJHM L R< JRWYW,RGLJHM,L R<,L H R</,,0.0,prep,,,det,,simple,,,qal,impf,,,L,,,,,,,,
4,347076,MHR[,MT,Proverbs,1,16,JMHRW,KJ RGLJHM L R< JRWYW W JMHRW L CPK DM,W JMHRW,,no complement,no complement,,,,,,,,,,,piel,impf,,,,,,,,,,,
5,347143,CWB[,MT,Proverbs,1,23,TCWBW,TCWBW L TWKXTJ HNH >BJ<H LKM RWXJ >WDJ<H DBRJ ...,TCWBW L TWKXTJ,,L TWKXTJ,L TWKXT/,,0.0,prep,,,det,subs,complex,,,qal,impf,,,L,,,,,,,,
6,347177,BW>[,MT,Proverbs,1,26,B>,GM >NJ B >JDKM >FXQ >L<G B B> PXDKM,B B> PXDKM,PXDKM,no complement,no complement,,,,,,,,,,,qal,infc,,,,,,,,,,,
7,347180,BW>[,MT,Proverbs,1,27,B>,B B> K C>WH PXDKM W >JDKM K SWPH J>TH B B> <LJ...,B B> K C>WH PXDKM,PXDKM,no complement,no complement,,,,,,,,,,,qal,infc,,,,,,,,,,,
8,347188,>TH[,MT,Proverbs,1,27,J>TH,B B> K C>WH PXDKM W >JDKM K SWPH J>TH B B> <LJ...,W >JDKM K SWPH J>TH,>JDKM,no complement,no complement,,,,,,,,,,,qal,impf,,,,,,,,,,,
9,347190,BW>[,MT,Proverbs,1,27,B>,B B> K C>WH PXDKM W >JDKM K SWPH J>TH B B> <LJ...,B B> <LJKM YRH W YWQH,YRH W YWQH,<LJKM,<L,,0.0,prep,prsf,,det,prsf,simple,,,qal,infc,,,<L,,,,,,,,


In [49]:
df.to_csv(PATH, index=False)